In [104]:
from DataProcessing import DataProcessing
from Car import Car
from Cba import Cba
from RuleItem import RuleItem

In [105]:
import pandas as pd
import time

In [110]:
data_procesing = DataProcessing("adult.data")
data_procesing.read_data()
data_procesing.process()
data = data_procesing.encoded_data
MIN_SUPPORT = 0.1
MIN_CONFIDENCE = 0.7
car = Car(data, MIN_SUPPORT, MIN_CONFIDENCE)
car.generate_frequent()
car.sort_rule()
cba = Cba(data, car.rule)
cba.cover()

print("Final Rules:")
for rule in cba.final_rules:
    rule['rule'].print_ruleitem()
    print(f"Error: {rule['error']}")
    print("")
print(f"Default: {cba.default}")

KeyboardInterrupt: 